In [34]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import pygrib
import netCDF4
import scipy
import matplotlib.pyplot as plt
DATA_DIR="/root/data_downloads"
era5_multi_level_dir=f"{DATA_DIR}/era5/multi"
era5_single_level_dir=f"{DATA_DIR}/era5/single"


## Get get ibtracs labels for current year

In [3]:
s3key = "data/ibtracs/IBTrACS.since1980.v04r00.nc"
ibtracs_dir = f"{DATA_DIR}/ibtracs"
%alias mkdatadir mkdir -p %l
%mkdatadir $ibtracs_dir
# dest = "/data_downloads/ibtracs/IBTrACS.ALL.v04r00.nc"
ibtracs_dest = f"{ibtracs_dir}/IBTrACS.since1980.v04r00.nc"
if os.path.exists(ibtracs_dest):
    print(f"{ibtracs_dest} already downloaded")
else:
    s3 = boto3.client('s3')
    s3.download_file(bucket, s3key, ibtracs_dest)
    print(f"downloaded {ibtracs_dest}")

/root/data_downloads/ibtracs/IBTrACS.since1980.v04r00.nc already downloaded


In [45]:
ibtracs_nc = netCDF4.Dataset(ibtracs_dest)
"""
Need to combine the datasets by hour.  Will want to OR the storms together where they share hours 
to produce a global mask of TC labels.  This will then be included in the outputted netcdf files.
"""
def ibtracs_time_to_pd_timestamp(ibtracs_time):
    ts = pd.to_datetime(ibtracs_time)
    return ts


ibtracs_roci = ibtracs_nc.variables["bom_roci"]
start_year = 2018
end_year = 2018
timestamp_storms = {}
print_freq = 100
for storm_idx, storm_times in enumerate(ibtracs_nc.variables["iso_time"][:]):
    for time_idx, time in enumerate(storm_times):
        timestamp_str = time.tobytes().decode("utf-8")
        if not timestamp_str:
            break # we have hit the last timestamp for the storm
        # Convert byte arrays to pandas Timestamp
        timestamp = ibtracs_time_to_pd_timestamp(timestamp_str) # 
        if start_year <= timestamp.year <= end_year:
            storm_tup = (storm_idx, time_idx)
            if timestamp_str in timestamp_storms:
                timestamp_storms[timestamp_str].append(storm_tup)
            else:
                timestamp_storms[timestamp_str] = [storm_tup]
    if storm_idx % print_freq == 0:
        print(f"{storm_idx} of {ibtracs_nc.variables['iso_time'].shape[0]}")


0 of 4687
100 of 4687
200 of 4687
300 of 4687
400 of 4687
500 of 4687
600 of 4687
700 of 4687
800 of 4687
900 of 4687
1000 of 4687
1100 of 4687
1200 of 4687
1300 of 4687
1400 of 4687
1500 of 4687
1600 of 4687
1700 of 4687
1800 of 4687
1900 of 4687
2000 of 4687
2100 of 4687
2200 of 4687
2300 of 4687
2400 of 4687
2500 of 4687
2600 of 4687
2700 of 4687
2800 of 4687
2900 of 4687
3000 of 4687
3100 of 4687
3200 of 4687
3300 of 4687
3400 of 4687
3500 of 4687
3600 of 4687
3700 of 4687
3800 of 4687
3900 of 4687
4000 of 4687
4100 of 4687
4200 of 4687
4300 of 4687
4400 of 4687
4500 of 4687
4600 of 4687


In [31]:
print(f"Storms found at {len(timestamp_storms)} timestamps in the {start_year}-{end_year} year range")

Storms found at 2759 timestamps in the 2018-2018 year range


## Process ERA5 data and output augmented netcdf file for each day

In [7]:
year = "2018"
month = "01"
day = "01"
# filename=f'{era5_single_level_dir}/{year}-{month}-{day}.grib'
single_file=f'{era5_single_level_dir}/{year}-{month}-{day}.nc'
multi_file=f'{era5_multi_level_dir}/{year}-{month}-{day}.nc'

single_nc =  netCDF4.Dataset(single_file)
multi_nc =  netCDF4.Dataset(multi_file)
single_vars = single_nc.variables.keys()
multi_vars = multi_nc.variables.keys()


mtpr = xr.DataArray(single_nc.variables["mtpr"], coords={
    'time': single_nc.variables["time"][:],
    'lat': single_nc.variables["latitude"][:], 
    'lon': single_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

sp = xr.DataArray(single_nc.variables["sp"], coords={
    'time': single_nc.variables["time"][:],
    'lat': single_nc.variables["latitude"][:], 
    'lon': single_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

tcwv = xr.DataArray(single_nc.variables["tcwv"], coords={
    'time': single_nc.variables["time"][:],
    'lat': single_nc.variables["latitude"][:], 
    'lon': single_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

pv850 = xr.DataArray(multi_nc.variables["pv"], coords={
    'time': multi_nc.variables["time"][:],
    'lat': multi_nc.variables["latitude"][:], 
    'lon': multi_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

u850 = xr.DataArray(multi_nc.variables["u"], coords={
    'time': multi_nc.variables["time"][:],
    'lat': multi_nc.variables["latitude"][:], 
    'lon': multi_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

v850 = xr.DataArray(multi_nc.variables["v"], coords={
    'time': multi_nc.variables["time"][:],
    'lat': multi_nc.variables["latitude"][:], 
    'lon': multi_nc.variables["longitude"][:]}, 
    dims=('time','lat', 'lon'))

ws850 = (u850**2 + v850**2)**0.5
ws850.rename("ws850")

<xarray.DataArray 'ws850' (time: 8, lat: 721, lon: 1440)>
array([[[4.19799256e-04, 4.19799256e-04, 4.19799256e-04, ...,
         4.19799256e-04, 4.19799256e-04, 4.19799256e-04],
        [2.66813476e+00, 2.66943138e+00, 2.66747718e+00, ...,
         2.67090597e+00, 2.67140183e+00, 2.66935204e+00],
        [2.54055172e+00, 2.53940546e+00, 2.54004819e+00, ...,
         2.54046987e+00, 2.54105267e+00, 2.54075952e+00],
        ...,
        [6.80254667e+00, 6.80748234e+00, 6.81183096e+00, ...,
         6.78878133e+00, 6.79374808e+00, 6.79843407e+00],
        [6.91538046e+00, 6.91838209e+00, 6.92264045e+00, ...,
         6.90575423e+00, 6.90899205e+00, 6.91283071e+00],
        [4.19799256e-04, 4.19799256e-04, 4.19799256e-04, ...,
         4.19799256e-04, 4.19799256e-04, 4.19799256e-04]],

       [[4.19799256e-04, 4.19799256e-04, 4.19799256e-04, ...,
         4.19799256e-04, 4.19799256e-04, 4.19799256e-04],
        [3.14505051e+00, 3.14574767e+00, 3.14738407e+00, ...,
         3.14046141e+00, 3.14317573e+00, 3.14470435e+00],
        [3.57811820e+00, 3.57950765e+00, 3.58124600e+00, ...,
         3.57384246e+00, 3.57528209e+00, 3.57675832e+00],
...
        [6.84448454e+00, 6.84652590e+00, 6.85022726e+00, ...,
         6.83490014e+00, 6.83810463e+00, 6.84103059e+00],
        [7.09789217e+00, 7.10007432e+00, 7.10416579e+00, ...,
         7.09151954e+00, 7.09330744e+00, 7.09696665e+00],
        [1.34494622e-03, 1.34494622e-03, 1.34494622e-03, ...,
         1.34494622e-03, 1.34494622e-03, 1.34494622e-03]],

       [[8.69204038e-04, 8.69204038e-04, 8.69204038e-04, ...,
         8.69204038e-04, 8.69204038e-04, 8.69204038e-04],
        [3.23552685e+00, 3.23403512e+00, 3.23760034e+00, ...,
         3.22961288e+00, 3.23128702e+00, 3.23207364e+00],
        [3.16790762e+00, 3.17126416e+00, 3.17381110e+00, ...,
         3.15717341e+00, 3.15957428e+00, 3.16457781e+00],
        ...,
        [5.85794561e+00, 5.86226013e+00, 5.86805802e+00, ...,
         5.84532152e+00, 5.84906735e+00, 5.85458832e+00],
        [6.01027009e+00, 6.01515321e+00, 6.01785858e+00, ...,
         6.00332889e+00, 6.00558483e+00, 6.00778707e+00],
        [8.69204038e-04, 8.69204038e-04, 8.69204038e-04, ...,
         8.69204038e-04, 8.69204038e-04, 8.69204038e-04]]])
Coordinates:
  * time     (time) int32 1034376 1034379 1034382 ... 1034391 1034394 1034397
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8

## Compute labels

In [ ]:
def haversine_distance(lat, lon, lat_center, lon_center):
    """
    returns matrix of dim (lat,lon) with distance in nautical miles
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat, lon, lat_center, lon_center])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat[:,None] / 2) ** 2 + np.cos(lat1[:,None]) * np.cos(lat2) * np.sin(dlon[None,:] / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of the Earth in kilometers

    # Calculate the distance in 
    distance = c * r
    # 1.852 km per nautical mile
    return distance / 1.852



# max_roci = np.argmax(ibtracs_nc.variables["bom_roci"])
# max_roci = np.unravel_index(max_roci, ibtracs_nc.variables["bom_roci"].shape)
# print("roci max", np.max(ibtracs.variables["bom_roci"]), max_roci)


# need to treat fill values as none or numpy gets unhappy
masked_lat = ibtracs_nc.variables["lat"][:]
masked_lat = np.ma.masked_array(masked_lat, mask=(masked_lat == ibtracs_nc.variables["lat"]._FillValue), fill_value=None)
masked_lon = ibtracs_nc.variables["lon"][:]
masked_lon = np.ma.masked_array(masked_lon, mask=(masked_lon == ibtracs_nc.variables["lon"]._FillValue), fill_value=None)
masked_roci = ibtracs_nc.variables["bom_roci"][:]
masked_roci = np.ma.masked_array(masked_roci, mask=(masked_roci == ibtracs_nc.variables["bom_roci"]._FillValue), fill_value=None)


# generate a mask per storm
print(tc_labels.shape) 
print(single_nc.variables["latitude"].shape)
print_freq = 100
iteration = 0
for timestamp, storms in timestamp_storms.items():
    timestep_labels = np.zeros((single_nc.dimensions["latitude"].size, single_nc.dimensions["longitude"].size), dtype=np.int8)
    for storm_id,timestep in storms:
        

        
        distances = haversine_distance(
            single_nc.variables["latitude"], # pull out the time  
            single_nc.variables["longitude"], 
            masked_lat[storm_id,timestep], 
            masked_lon[storm_id,timestep]
        )
        storm_labels = np.where(distances <= masked_roci[storm_id,timestep], 1, 0)
        timestep_labels = timestep_labels | storm_labels
    if iteration % print_freq == 0:
        print(f"{iteration} of {len(timestamp_storms)}")
    iteration += 1
    # print(np.max(timestep_labels))
    
# # storm_id = 2403
# timestep = 5

# distances = haversine_distance(era5_single.variables["latitude"], era5_single.variables["longitude"], masked_lat[storm_id,timestep], masked_lon[storm_id,timestep])

# # print(distances.shape) 
# tc_label = np.where(distances <= masked_roci[storm_id,timestep], 1, 0)

(721, 1440)
(721,)
0 of 2759


In [ ]:

tcwv.isel(time=0).plot.contourf()
plt.show()
pv850.isel(time=0).plot.contourf()
plt.show()
mtpr.isel(time=0).plot.contourf()
plt.show()
ws850.isel(time=0).plot.contourf()
plt.show()
# plt.colorbar()
